###### Import the Data

Our first step is going to be to import data. We have three dataset options to choose from in the kaggle link - IMDb, Amazon or Yelp. We are going to select IMDb.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

data= pd.read_csv('/content/imdb_labelled.txt', names=['Reviews','Sentiment'], delimiter = '\t')
